# Мешок слов и классификация текстов

Одна из базовых задач машинного обучения - это классификация (отнесение объекта к одному из заранее заданных классов). Классификация очень хорошо подходит для задач NLP, так как многие практические задачи можно свести к классификации (определение спама, определение тональности, определение намерения пользователя и т.п.).

Есть очень много алгоритмов МО, которые применимы к NLP. У каждого свои плюсы, минусы и особенности, которые по-разному проявляются на практике. 

За одно занятие разобрать все важные алгоритмы не получится, поэтому сегодня поговорим про алгоритмы, которые из-за простоты использования удобно применять в качестве начальной базовой (baseline) модели. Более сложные алгоритмы (бустинги, lstm, трансформеры), практически всегда, будут работать лучше, НО помимо качества в практических задачах часто есть много других требований и ограничений (скорость, память, интерпретируемость и тп), которые могут перевесить разницу в точности. 

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

from IPython.display import Image
from IPython.core.display import HTML 

В этой тетрадке мы рассмотрим задачу определения токсичных твитов. 

In [2]:
data = pd.read_csv('labeled.csv')

In [3]:
data.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14412 entries, 0 to 14411
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   comment  14412 non-null  object 
 1   toxic    14412 non-null  float64
dtypes: float64(1), object(1)
memory usage: 225.3+ KB


В определении тональности токсичных текстов обычно значительно меньше. Но даже сильный дисбаланс классов (99 к 1,например) - это часто не проблема для алгоритма/модели, нужно лишь быть аккуратным с оцениванием.

In [4]:
data.toxic.value_counts(normalize=True)

toxic
0.0    0.66514
1.0    0.33486
Name: proportion, dtype: float64

Для того, чтобы понять качество обученной модели нам нужно разделить данные на обучающую и тестовую выборку. Оценивать модель нужно всегда на данных, которые модель еще не видела! 

In [5]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)

In [12]:
train.shape

(12970, 3)

In [6]:
train.reset_index(inplace=True)
test.reset_index(inplace=True)

### Векторизация текста

Перед тем, как рассматривать алгоритмы классификации, нужно разобраться с тем, как мы будем подавать наши тексты на вход разным алгоритмам. Все они работают с числами, и текст, соответственно, тоже нужно превратить в число. А точнее в набор чисел - вектор. 

Самый простой способ векторного представления текста называется "мешком слов" (bag-of-words). Мешок тут не какой-то технический термин, а метафора. В таком способе векторизации никак не учитывается порядок. Слова как бы складываются в "мешок" и перемешиваются. 

Если более формально, то для того, чтобы векторизовать некоторый набор документов (=текстов) мешком слов нужно:  
а) составить словарь всех уникальных слов, встречаемых в этих документах 
б) зафиксировать порядок слов в словаре и сопоставить каждому из них порядковый индекс
б) составить для каждого документа вектор размерности N (N - равен размеру словаря), где по индексу i стоит частота слова w_i в этом документе. 

Вот картинка для наглядности:

In [7]:
Image(url="https://i.ibb.co/r5Nc2HC/abs-bow.jpg",
     width=500, height=500)

Обратите внимание на то, что порядок колонок (слов в словаре) в этой таблице не соответствует порядку слов в текстах.

На картинке выше частотность записана в абсолютных значениях. Лучше использовать относительную частоту, чтобы тексты разной длины были сопоставимы друг с другом.

In [8]:
Image(url="https://i.ibb.co/47bRcVy/bow-normalized.jpg",
     width=500, height=500)

CountVectorizer в sklearn векторизует как раз таким образом

In [9]:
vectorizer = CountVectorizer()
# в векторайзер нужно засовывать тексты строками (токенизация там встроена)
X = vectorizer.fit_transform(train.comment)

### TF-IDF

Еще вместо частот можно использовать tf-idf (term frequency - inverse document frequency). 

Кратко про tfidf на картинке

In [10]:
Image(url="https://miro.medium.com/max/1200/1*V9ac4hLVyms79jl65Ym_Bw.jpeg",
     width=600, height=600)

Tfidf позвозволяет оштрафовать слова, которые встречаются в большом количестве документов (грубо говоря это стоп-слова, но специфичные для корпуса) и поднять важность слов, которые встречаются часто в небольшом количестве документов. Если слово встречается во всех документах, то соотношение $N/df_x$ будет равно 1, а логарифм от 1 - 0. Чем меньше $df_x$, тем больше будет $log(N/df_x)$

А в вектор таким образом добавляется информация обо всем корпусе. Обычно для модели это оказывается полезно.

In [9]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(train.comment)

Векторайзер возвращает результат в виде матрицы

In [10]:
X.shape
# первая размерность - количество документов
# вторая размерность - количесто слов в словаре

(12970, 64361)

In [15]:
# vectorizer.get_feature_names_out()

Слов получается в 5 раз больше, чем документов. Некоторые алгоритмы не смогут так обучиться (нужно чтобы признаки <= документы), а те что смогут будут обучаться сильно дольше. И учитывая, что большая часть слов встретились по 1 разу, они все равно никак не помогут.

In [27]:
vectorizer = TfidfVectorizer(min_df=5, max_df=0.4)
X = vectorizer.fit_transform(train.comment)

In [28]:
X.shape

(12970, 7640)

В словаре у нас 7,5 тысяч слов  и каждый документ мы описываем вектором такого размера. В этом векторе положительными будут значения, соответствующие словам, которые есть в этом тексте, а нулевыми - те значения, которые соответствуют словам, которых в тексте нет.

**Нулевых значения будет значительно больше!**
Поэтому для эффективности в sklearn такие матрицы хранятся в специальном sparse (разреженном) формате.

Просто взять и посмотреть на матрицу не получится.

In [17]:
X # 202052 из 98027260 позиций в матрице ненулевые (это меньше 1 процента)

<12970x7640 sparse matrix of type '<class 'numpy.float64'>'
	with 202989 stored elements in Compressed Sparse Row format>

Некоторые алгоритмы не умеют работать с разреженным векторами и такую матрицу можно привести в обычный dense формат (X.todense()) НО будьте острожны с большими матрицами - они будут занимать в памяти ОЧЕНЬ много места.

### Косинусная близость

Между получившимеся векторами можно посчитать близость используя косинусное расстояние. 

Подробное описание того, как работает косинусное расстояние есть в [3-ем подготовительном семинаре](https://github.com/mannefedov/compling_nlp_hse_course/blob/master/notebooks/first_module_intro/03_lexical_disambiguation.ipynb)

Для bow-векторов косинусное расстояние главным образом будет зависеть от количества общих слов в двух документах. 

Давайте посмотрим на настоящих текстах текстами.

In [35]:
train.loc[3, 'comment']

'Получается, у вас либеральное обострение?\n'

В sklearn есть косинусное расстояние и косинусная близость. Расстояние это просто единица минус близость (и наоборот), то есть расстояние между близкими векторами должно быть маленькое (0 если совпадают 1 - если вообще не совпадают), а близость наоборот (1 если совпадают 0 если не совпадают совсем).

Расстояние удобнее использовать, когда нужно отсортировать по близости, т.к в numpy по умолчанию сортируется по возрастанию.

In [37]:
# функция предназначена для расчета близости между массивами векторов
# и возвращает она тоже массив где каждая строчка это объект и первого массива, 
# а каждая колонка это близость до объекта во втором массиве

# в нашем случае в первом массиве у нас только 1 вектор
# поэтому мы можем взять первую строчку из получившегося массива
# метод .argsort вернет список индексов по возрастанию 
# возьмем первые три индекса и посмотрим что там за тексты
top_idx = cosine_distances(X[3], X).argsort()[0,:3]
top_idx

array([    3, 12387,  4296])

In [38]:
# первым нашелся этот же вектор
# а дальше уже не настолько близкие но все равно есть сходство
train.loc[top_idx, 'comment'].values

array(['Получается, у вас либеральное обострение?\n',
       'У вас что, ночное большевистское обострение?',
       'У вас что, ночное большевистское обострение?\n'], dtype=object)

Мы векторизовали обучающую выборку, осталось векторизовать тестовую

Векторайзеры в sklearn имеют три основных метода **fit**, **transform** и **fit_transform**. 

**fit** - собирает словарь и статистики по текстам,   
**transform** - преобразует тексты в векторы, на основе уже собранного словаря.  
**fit_transform** - делает сразу и первое и второе (быстре чем 1 и 2 по очереди).

Для теста нам нужно векторизовать тексты тем же словарем, для этого вызовем метод .transform

Повторим еще раз векторизацию и достанем отдельно целевую переменную.

In [46]:
vectorizer = TfidfVectorizer(min_df=10, max_features=100, max_df=0.3)

X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment) 

In [47]:
X.shape, X_test.shape

((12970, 100), (1442, 100))

In [48]:
y = train.toxic.values
y_test = test.toxic.values

In [49]:
Y,

(<12970x100 sparse matrix of type '<class 'numpy.float64'>'
 	with 70698 stored elements in Compressed Sparse Row format>,
 array([1., 1., 0., ..., 1., 0., 1.]))

In [139]:
# ! Называть матрицу с признаками X, а вектор с целевой переменной y - стандартная практика. 
# ! Если вы сделаете наоборот, то сильно запутаете меня в домашке 


Интуитивно кажется, что bag-of-words сильно упрощает - порядок слов ведь очень важен для текста (целое больше суммы отдельных элементов, синергия, эмерджентность и все такое)! Однако на практике BOW работает удивительно хорошо. Начать решать практически любую классификационную нлп задачу лучше всего с мешка слов и простого алгоритма классификации. Это хорошее базовое решение, с которым удобно сравнивать более сложные решения и к тому же есть шанс, что уже такого базового решения будет достаточно для практического применения.


## Алгоритмы классификации

Теперь попробуем обучить модель. Рассмотрим 5 алгоритма, которые можно попробовать прежде чем переходить к чему-то более сложному: KNN, Logistic Regression, Decision Trees, Naive Bayes, RandomForest.

Кратко идея этих алгоритмов:

1) **KNN** - это предсказания класса текста по близости к другим текстам, для которых известен класс. Выше мы считали близость между текстами косинусным расстоянием - в KNN делается то же самое (но метрика может быть другая), только рассматривается топ-K ближайших текстов. В KNN таким образом нет никакого обучения - просто запоминание тестовой выборки и сравнение с ней при предсказании.

На графике можно нарисовать зоны, которые будут соответствовать классам для выбраного параметра K. 

![источник:https://mlarchive.com/machine-learning/k-nearest-neighbor-knn-explained/](https://mlarchive.com/wp-content/uploads/2022/09/img2-3-1024x585.png)

2) **Логистическая регрессия.** Предсказание с помощью обученной модели логистической регрессии - это по сути взвешенное средние чисел во входном векторе и приведение получившегося числа в интервал от 0 до 1 с помощью специальной функции (сигмоиды). Если получившееся значение больше 0.5, то считаем, что этот текст "токсичный", а если нет, то "нейтральный" (но можем подобрать и другие пороги).

Коэффициенты подбираются на обучающих данных (это и есть обучение). Можно сказать, что в итоге для каждого слова находится показатель токсичности. Если в тексте будет много слов с высоким показателем токсичности, то весь текст будет отнесен к токсичному классу. Однако показатель токсичности не равно вероятность токсичности - он может быть равен любому числу (например, 0.282, -4815162342, 666.13) Интерпретировать значение показателя можно только по отношению к другим значениям.

Коэффициенты обученной модели будут являться уравнением прямой, которая разделяет объекты на классы. И эту прямую можно нарисовать, но с оговорками. Во-первых, больше 2 признаков нарисовать не получится, а во вторых, так как это не линейная регрессия, где на выходе мы получаем число, прямая для логистической регрессии находится в другом пространстве, из которого можно перейти к классовым вероятностям с помощью сигмоиды. (можно посмотреть серию видео вот тут, чтобы копнуть поглубже - https://www.youtube.com/watch?v=yIYKR4sgzI8&list=PLblh5JKOoLUKxzEP5HA2d-Li7IJkHfXSe )


In [4]:
# так примерно выглядит прямая регрессии для двух признаков (класс показан цветом точек)
# прямая тут показывает разделяющую поверхность
# можно взять какой-то x1 и x2 и посмотреть с какой стороны от разделяющей поверхности находится точка 
# и понять какой класс будет предсказан

![](https://scipython.com/static/media/uploads/blog/logistic_regression/decision-boundary.png)

Легче не пытаться визуализировать логистическую регресиию, а посмотреть как она рассчитывается и что на что в итоге влияет.

Каждый признак в inputs умножается на свой коэффициент, они складываются и пропускаются через сигмоиду, которая преобразует сумму в вероятность от 0 до 1. 

In [30]:
Image(url="https://deeplearningmath.org/images/shallow_NN.png",
     width=700, height=700)

3) **Дерево решений** - это просто много вложенных if else. В процессе обучения подбирается такая серия условий типа "тфидф этого слова больше 1.23" или "частотность слова_а больше частотности слова_б", чтобы в итоге получалось предсказать правильный класс текста. Плюсом деревьев решений является их интерпретируемость - для каждого предсказания можно вывести цепочку условий, которая привела к такому выводу. 

Каждое условие в дереве разделяет пространство пополам, поэтому график решающего дерева будет состоять из четырехугольников.

In [7]:
Image(url="https://paulvanderlaken.files.wordpress.com/2020/03/readme-titanic_plot-11.png",
     width=700, height=700)

4) **Наивный байесовский классификатор**. НБ классификатор принимает решение по вероятностям, рассчитаным на обучающем корпусе. В частности нужно три вероятности:  
а) вероятность встретить конкретное слово в классе А и Б (токсик и нетоксик в нашем случае);   
б) вероятность каждого класса.
в) вероятность каждого слова.

Эти вероятности расчитываются по частностям. Например, количество токсичных документов со словом, поделенное на общее количество документов с этим словом - это вероятность встретить слово в токсичном классе, а количество токсичных документов, поделенное на общее количество документов - это вероятность токсичного класса, а количество вхождений слова, поделенное на общее количество слов - вероятность слова.

Эти вероятности перемножаются по формеле Баеса. 

In [28]:
# P(B|A) - вероятность встретить конкретное слово B в классе A
# P(A) - вероятность класса А
# P(B) - вероятность слова B
Image(url="https://media.proglib.io/posts/2021/10/18/4956938de749b78809ab37725a14cb52.png",
     width=500, height=500)


Можно сказать, что в наивном байесе ничего и не обучается, а просто считаются статистики. При предсказании для каждого класса перемножаются соответствующие вероятности слов, присуствующих в тексте, и класс набравший большую оценку выбирается. 

Наивным такой классификатор называется, потому что делается предположение, что слова в документе не зависят друг от друга. Это конечно не так, многие слова появляются в тексте вместе не случайно. Допустим, мы рассматриваем тексты на английском. У нас могут быть тексты про яблоки (apples), про компанию Apple, и тексты про Нью-Йорк (Big Apple). После нормализации в словарь во всех трех случаях попадет "apple" и вероятность будет считаться по всем типам текстов. Но возможна такая ситуация, что тексты про яблоки нейтральные, про компанию - негативные, а про Нью-Йорк - положительные. Если бы мы считали совместные вероятности слов, то мы смогли разделить эти типы и делать более точные предсказания. Но это сильно сложнее, а наивный байес работает достаточно хорошо и так. 

5) **RandomForest** - это N деревьев решений, объединенных в один большой классификатор (лес). Такие классификаторы называются ансамблями. Ставить RF в один ряд с алгоритмами выше наверное не очень корректно - RF сильно сложнее и мощнее, но на практике его использование в sklearn никак не отличается, нужно только немного разобраться с основыми параметрами. 

Для каждого отдельного дерева в RF используется случайная подвыборка обьектов (текстов) и признаков (слов), чтобы деревья не получались одинаковыми. Поэтому алгоритм и называется случайным лесом. Отдельные деревья решений могут быть очень слабыми, если пробовать применить их самостоятельно, но вместе они дают очень хороший результат. 

In [32]:
# суть Random Forest
Image(url="https://www.meme-arsenal.com/memes/b21eaf3fcba087cc3dc2e6c43eaa7eeb.jpg",
     width=500, height=500)

### Fit, predict

У алгоритмов в sklearn стандартный интерфейс - есть функции fit, predict и predict_proba (если классификатор выдает вероятности)

In [54]:
clf = LogisticRegression(C=0.1, class_weight='balanced')

In [51]:
# fit обучает модель
clf.fit(X, y)

LogisticRegression(C=0.1, class_weight='balanced')

In [52]:
clf.classes_

array([0., 1.])

In [53]:
# предикт предсказывает классы
preds = clf.predict(X_test)

In [57]:
preds

array([0., 0., 1., ..., 0., 1., 1.])

In [59]:
y_test

array([0., 0., 1., ..., 1., 1., 0.])

In [60]:
sum(preds == y_test) / len(y_test)

0.710124826629681

### Метрики

#### Меры качества бинарной классификации 

При сопоставлении предсказаний модели с правильными (истинными) ответами составляют вот такую таблицу

<table>
  <tr>
    <th colspan="2"</th>
    <th colspan="2">правильные <br>ответы</th>
  </tr>
  <tr>
    <td></td>
    <td></td>
    <td>positive</td>
    <td>negative</td>
  </tr>
  <tr>
    <td rowspan="2">предсказания <br>модели</td>
    <td>positive</td>
    <td><span style="color:green">$tp$</span></td>
    <td><span style="color:red">$fp$</span></td>
  </tr>
  <tr>
    <td>negative</td>
    <td><span style="color:red">$fn$</span></td>
    <td><span style="color:green">$tn$</span></td>
  </tr>
</table>


$tp$ - это $true\_positive$, количество истинно положительных предсказаний модели (т.е. модель считает, что текст токсичный и он на самом деле токсичный)

$fp$ - это $false\_positive$, количество ложно положительных предсказаний модели (т.е. модель считает, что текст токсичный, а он на самом деле НЕ токсичный)

$tn$ - это $true\_negative$, количество истинно отрицательных предсказаний модели (т.е. модель считает, что текст нетоксичный и он на самом деле нетоксичный)

$fn$ - это $false\_negative$, количество ложно отрицательных предсказаний модели (т.е. модель считает, что текст нетоксичный и он на самом деле токсичный)

Зеленым в таблице выделены правильные ответы, а красным - ошибки. Часто их называют ошибка первого (false positive) и второго рода (false negative).

Для оценки моделей эти значений обычно не используются. На их основе считаются другие метрики. Основные для нас это: accuracy, точность, полнота и f-мера:

$precision = Pr =  \frac{tp}{tp+fp} $ – точность (показывает, какая доля положительных предсказаний является правильными)

$recall = R = \frac{tp}{tp+fn} $ – полнота (показывает, насколько полно предсказывается положительный класс)

$F_1 = \frac{2 Pr * R}{Pr + R}$ – $F$-мера (объединяет точность и полноту в одно число)

$accuracy = \frac{tp + tn}{tp + fp + fn + tn}$ –  по-русски это тоже точность, поэтому иногда возникает путаница; лучше всего говорить accuracy (это просто доля правильных ответов)

Accuracy хоть и самая простая и логичная, но используется редко. Эта метрика не устойчива к дисбалансу классов, а он почти всегда есть.

По умолчанию лучше всего смотреть на F-меру. Но бывают задачи, где важнее точность, а бывают - где важнее полнота. Например, если на основе предсказания токсичности автору сразу дается бан, то нам нужно быть увереным, что эти предсказания являются очень точными. А если мы хотим снизить количество токсичности на платформе до минимума, то нам будет важнее полнота.

В sklearn есть отдельные функции для каждой из метрик, а есть удобная функция classification_report, которая считает все сразу

In [162]:
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.86      0.82      0.84       952
         1.0       0.68      0.75      0.71       490

    accuracy                           0.80      1442
   macro avg       0.77      0.78      0.78      1442
weighted avg       0.80      0.80      0.80      1442



In [163]:
# predict_proba возвращает вероятности классов
# это полезно когда нужно подобрать порог например
probas = clf.predict_proba(X_test)

In [164]:
# в левой колонке вероятность 0 (нетокстичности)
# в правой - вероятность 1 (токсичности)
probas

array([[0.80526376, 0.19473624],
       [0.10602066, 0.89397934],
       [0.50724877, 0.49275123],
       ...,
       [0.62165136, 0.37834864],
       [0.52490349, 0.47509651],
       [0.29114768, 0.70885232]])

In [167]:
# возьмем вторую колонку
# проверим что она больше 0,85
# заменим True и False на 0 и 1 чтобы получить предсказания
preds = (probas[:,1]>0.85).astype(int)

In [168]:
# точность по токсичному классу сильно выросла но при этом упала полнота
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.67      1.00      0.80       952
         1.0       0.95      0.04      0.07       490

    accuracy                           0.67      1442
   macro avg       0.81      0.52      0.44      1442
weighted avg       0.76      0.67      0.55      1442



Попробуем другие классификаторы

In [170]:
clf = MultinomialNB(alpha=1.)
clf.fit(X, y)
preds = clf.predict(X_test)

print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.80      0.97      0.88       952
         1.0       0.89      0.54      0.67       490

    accuracy                           0.82      1442
   macro avg       0.85      0.75      0.77      1442
weighted avg       0.83      0.82      0.81      1442



In [171]:
clf = DecisionTreeClassifier(max_depth=8, class_weight='balanced')
clf.fit(X, y)
preds = clf.predict(X_test)

print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.82      0.37      0.51       952
         1.0       0.41      0.84      0.55       490

    accuracy                           0.53      1442
   macro avg       0.61      0.61      0.53      1442
weighted avg       0.68      0.53      0.53      1442



In [172]:
clf = KNeighborsClassifier(n_neighbors=10, metric='cosine')
clf.fit(X, y)
preds = clf.predict(X_test)

print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.80      0.85      0.82       952
         1.0       0.67      0.60      0.63       490

    accuracy                           0.76      1442
   macro avg       0.73      0.72      0.73      1442
weighted avg       0.76      0.76      0.76      1442



#### Меры качества многоклассовой классификации 
Если количество классов больше, то расчет метрик становится немного сложнее. Можно свести все к бинарной оценке, если рассматривать каждый класс по отношению ко всем другим классам. Т.е. считать текущий класс положительным, а все другие классы отрицательным классом. Для трех классов у нас получится три таблицы:

<table>
  <tr>
    <th colspan="4">правильные <br>ответы</th>
  </tr>
  <tr>
    <td></td>
    <td></td>
    <td>class 1</td>
    <td>(class 2, class 3)</td>
  </tr>
  <tr>
    <td rowspan="2">предсказания <br>модели</td>
    <td>class 1</td>
    <td><span style="color:green">$tp_1$</span></td>
    <td><span style="color:red">$fp_1$</span></td>
  </tr>
  <tr>
    <td>(class 2, class 3)</td>
    <td><span style="color:red">$fn_1$</span></td>
    <td><span style="color:green">$tn_1$</span></td>
  </tr>
</table>
<table>
  <tr>
    <th colspan="4">правильные <br>ответы</th>
  </tr>
  <tr>
    <td></td>
    <td></td>
    <td>class 2</td>
    <td>(class 1, class 3)</td>
  </tr>
  <tr>
    <td rowspan="2">предсказания <br>модели</td>
    <td>class 2</td>
    <td><span style="color:green">$tp_2$</span></td>
    <td><span style="color:red">$fp_2$</span></td>
  </tr>
  <tr>
    <td>(class 1, class 3)</td>
    <td><span style="color:red">$fn_2$</span></td>
    <td><span style="color:green">$tn_2$</span></td>
  </tr>
</table>
<table>
  <tr>
    <th colspan="4">правильные <br>ответы</th>
  </tr>
  <tr>
    <td></td>
    <td></td>
    <td>class 3</td>
    <td>(class 1, class 2)</td>
  </tr>
  <tr>
    <td rowspan="2">предсказания <br>модели</td>
    <td>class 3</td>
    <td><span style="color:green">$tp_3$</span></td>
    <td><span style="color:red">$fp_3$</span></td>
  </tr>
  <tr>
    <td>(class 1, class 2)</td>
    <td><span style="color:red">$fn_3$</span></td>
    <td><span style="color:green">$tn_3$</span></td>
  </tr>
</table>

Соответственно для каждого класса мы можем отдельно посчитать точность, полноту и F-меру. 
А если мы хотим найти общие метрики, то у нас есть два варианта: 

1) усреднить отдельные точности, полноты, F-меры по количеству классов (макро усреднение)

$macro\ Precision = \frac{\sum Pr_c}{|C|}$  
$macro\ Recall = \frac{\sum R_c}{|C|}$  
$macro\ F1\_measure = \frac{\sum F1_c}{|C|}$  

2) рассчитать общие tp, tn, fp, fn и вычислить точность, полноту по ним (микро усреднение)


$micro\ Precision =  \frac{\sum tp_c}{\sum tp_c+fp_c} $ 

$micro\ Recall = \frac{\sum tp_c}{\sum tp_c+fn_c} $ 

Микро усреднение зависит от баланса классов - доминирующие классы будут перетягивать метрику на себя, за счет того, что их tp,fp,tn,fn будут иметь больший вклад в результат. Макро усреднение не учитывает размер классов, что может привезти к тому, что мелкие классы, плохо классифицируемые моделью, будут непропорционально занижать общий результат. 

Макро усреднение можно улучшить дополнительным взвешиванием каждого класса. Можно взвесить на частотность (тогда это будет практически микро среднее), а можно придумать каждому классу свой вес, в зависимости от задачи.

Возьмем данные с несколькими классами и попробуем

In [35]:
data_multiclass = pd.read_csv('tweet_emotions.csv.zip')

In [36]:
data_multiclass.shape

(40000, 3)

In [175]:
counts = data_multiclass.sentiment.value_counts(normalize=True)

In [176]:
counts

neutral       0.215950
worry         0.211475
happiness     0.130225
sadness       0.129125
love          0.096050
surprise      0.054675
fun           0.044400
relief        0.038150
hate          0.033075
empty         0.020675
enthusiasm    0.018975
boredom       0.004475
anger         0.002750
Name: sentiment, dtype: float64

В данных есть совсем маленькие классы, заменим их на 1 общий класс other

In [177]:
data_multiclass.loc[data_multiclass.sentiment.isin(counts[counts<0.05].index), 'sentiment'] = 'other' 

In [178]:
train, test = train_test_split(data_multiclass, 
                               test_size=0.1, 
                               stratify=data_multiclass.sentiment,
                               shuffle=True)

train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [179]:
y = train.sentiment.values
y_test = test.sentiment.values

Обучаем какой-нибудь классификатор.

In [184]:
vectorizer = TfidfVectorizer(max_features=1000, min_df=5, max_df=0.4)
X = vectorizer.fit_transform(train.content)
X_test = vectorizer.transform(test.content) 

In [187]:
rf = RandomForestClassifier(n_estimators=100, max_depth=20, )
rf.fit(X, y)

preds = rf.predict(X_test)

In [188]:
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

   happiness       0.42      0.15      0.22       521
        love       0.53      0.33      0.41       384
     neutral       0.29      0.72      0.42       864
       other       0.40      0.06      0.11       650
     sadness       0.50      0.10      0.17       516
    surprise       0.00      0.00      0.00       219
       worry       0.32      0.48      0.39       846

    accuracy                           0.33      4000
   macro avg       0.35      0.26      0.24      4000
weighted avg       0.37      0.33      0.28      4000



## Мешок нграмм 

Для того, чтобы учесть информацию о порядке слов и при этом остаться в рамках простой мешкословной модели, можно использовать небольшой трюк - добавить в словарь нграммы.
 
О том, как найти хорошие нграммы, мы поговорим позже, а пока воспользуемся встроенными возможностями sklearn. У векторайзеров есть параметр ngram_range, который по умолчанию задан как (1,1). Мы можем изменить его на (1,2) или (1,3), чтобы в словаре появились еще биграммы и триграммы, соответственно.

In [5]:
data = pd.read_csv('labeled.csv')
train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [52]:
vectorizer = TfidfVectorizer(max_features=1000, min_df=10, max_df=0.1, ngram_range=(1, 3))
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment) 

In [53]:
y = train.toxic.values
y_test = test.toxic.values

Посмотрим, какие нграммы попали в словарь

In [54]:
#vectorizer.get_feature_names() показывает словарь, индексы в списке соответствуют колонкам в матрице
# нграмы склеиваются через пробел в sklearn
[x for x in vectorizer.get_feature_names() if ' ' in x][:30]

['10 лет',
 'больше чем',
 'бы не',
 'было бы',
 'во время',
 'вообще не',
 'вот это',
 'все же',
 'все равно',
 'все таки',
 'все это',
 'вы не',
 'где то',
 'да не',
 'даже если',
 'даже не',
 'для меня',
 'для того',
 'для этого',
 'до сих',
 'до сих пор',
 'думаю что',
 'если бы',
 'если не',
 'если ты',
 'если это',
 'же как',
 'же не',
 'за это',
 'зависит от']

In [57]:
rf = RandomForestClassifier(n_estimators=100, max_depth=20)
rf.fit(X, y)

preds = rf.predict(X_test)

print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.72      0.98      0.83       944
         1.0       0.88      0.29      0.43       498

    accuracy                           0.74      1442
   macro avg       0.80      0.63      0.63      1442
weighted avg       0.78      0.74      0.69      1442



## Стандартные датасеты для классификации

Выше все примеры были на датасете с анализом тональности. Давайте посмотрим на несколько других типичных датасетов для задачи классификации


Одним из самых широко используемых датасетов являются отзывы на imdb. Это тоже датасет для анализа тональности с бинарной классификацией, но его стоит упомянуть отдельно потому что он очень популярный

In [63]:
from datasets import load_dataset

In [2]:
dataset = load_dataset("stanfordnlp/imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [86]:
dataset['train'][100]

{'text': "Terrible movie. Nuff Said.<br /><br />These Lines are Just Filler. The movie was bad. Why I have to expand on that I don't know. This is already a waste of my time. I just wanted to warn others. Avoid this movie. The acting sucks and the writing is just moronic. Bad in every way. The only nice thing about the movie are Deniz Akkaya's breasts. Even that was ruined though by a terrible and unneeded rape scene. The movie is a poorly contrived and totally unbelievable piece of garbage.<br /><br />OK now I am just going to rag on IMDb for this stupid rule of 10 lines of text minimum. First I waste my time watching this offal. Then feeling compelled to warn others I create an account with IMDb only to discover that I have to write a friggen essay on the film just to express how bad I think it is. Totally unnecessary.",
 'label': 0}

Датасеты с отзывами вообще популярны. Часто в них еще бывает не бинарная классификация тональности, а категории (1-5 ил 1-10) 

Анализ тональности еще тесно связан с анализом токсичности. В таких текстах негативная тональность подразделяется на определенные типы (hateful, violence, harassment и т.д). Это немного разные задачи, так как негативная тональность не обязательно означает токсичность, но у этих задач есть большое пересечение

In [64]:
hatecheck = load_dataset('mteb/multi-hatecheck', )

README.md: 0.00B [00:00, ?B/s]

In [65]:
hatecheck

DatasetDict({
    test: Dataset({
        features: ['functionality', 'text', 'is_hateful', 'lang'],
        num_rows: 40310
    })
})

In [66]:
hatecheck['test'][0]

{'functionality': 'derog_neg_emote_h',
 'text': 'انا بكره النساء.',
 'is_hateful': 'hateful',
 'lang': 'ara'}

In [69]:
eng_hatecheck = hatecheck.filter(lambda e: e["lang"] == 'eng')

Filter:   0%|          | 0/40310 [00:00<?, ? examples/s]

In [74]:
eng_hatecheck["test"][1423]

{'functionality': 'ref_subs_clause_h',
 'text': "Disabled people are nothing like us and that's why they deserve to die. ",
 'is_hateful': 'hateful',
 'lang': 'eng'}

Другая типичная классификационная задача - анализ грамматичности текста. Есть стандартный датасет Cola, который очень часто используется для бенчмарка моделей

In [29]:
cola = load_dataset('nyu-mll/glue', 'cola')

Generating train split:   0%|          | 0/8551 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1043 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1063 [00:00<?, ? examples/s]

In [37]:
cola['train'][6]

{'sentence': 'Fred watered the plants flat.', 'label': 1, 'idx': 6}

Также типичная классификационная задача - предсказание намерения пользователя (intent detection). Она немного устарела с появлением хороших языковых моделей, но все равно хорошо подходит для бенчмарков

In [38]:
intent = load_dataset('mteb/amazon_massive_intent', "ru")

Resolving data files:   0%|          | 0/51 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/51 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/51 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [40]:
intent['train'][0]

{'id': '1',
 'label': 'alarm_set',
 'label_text': 'alarm_set',
 'text': 'разбуди меня в девять утра в пятницу',
 'lang': 'ru'}

Еще много классификационных датасетов можно найти тут например - 
https://embeddings-benchmark.github.io/mteb/overview/available_tasks/classification/ 

## Датасеты для парной классификации

И последнее о чем хочется сказать - это парная классификация. Это очень популярный тип датасетов, которые используются в том числе и в актуальных бенчмарках для языковых моделей. В таких датасетах вместо классификации одного текста нужно классифицировать пару текстов. Классическая задача для парной классификации - NLI (natural language entailment), где нужно предсказать следует ли второе предложение из второго или противоречит ему. 

Для такой задачи кончно нужны более серьезные модели, но и простой мешок слов тут тоже можно применить

Можно сделать два отдельных векторайзера для каждого из текстов в паре и перед обучением модели конкатенировать две матрицы в одну. Давайте попробуем это сделать на датасете MNLI

In [91]:
nli = cola = load_dataset('nyu-mll/glue', 'mnli')

In [56]:
nli['test_matched'][1]

{'premise': 'The extent of the behavioral effects would depend in part on the structure of the individual account program and any limits on accessing the funds.',
 'hypothesis': 'Many people would be very unhappy to loose control over their own money.',
 'label': -1,
 'idx': 1}

In [71]:
nli_train = nli['train'].select([i for i in range(5000)])
nli_test = nli['validation_matched'].select([i for i in range(1000)])

In [74]:
vectorizer_premise = TfidfVectorizer(max_features=1000, min_df=10, max_df=0.1, ngram_range=(1, 3))
vectorizer_hypothesis = TfidfVectorizer(max_features=1000, min_df=10, max_df=0.1, ngram_range=(1, 3))

X_premises = vectorizer_premise.fit_transform(nli_train['premise'])
X_hypothesis = vectorizer_hypothesis.fit_transform(nli_train['hypothesis'])


X_premises_test = vectorizer_premise.transform(nli_test['premise'])
X_hypothesis_test = vectorizer_hypothesis.transform(nli_test['hypothesis'])


In [75]:
from scipy.sparse import hstack

X = hstack([X_premises, X_hypothesis])
X_test = hstack([X_premises_test, X_hypothesis_test])

In [76]:
y = nli_train['label']
y_test = nli_test['label']

In [83]:
rf = RandomForestClassifier(n_estimators=200, max_depth=20)
rf.fit(X, y)

preds = rf.predict(X_test)

print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

           0       0.40      0.67      0.50       341
           1       0.53      0.15      0.23       319
           2       0.50      0.50      0.50       340

    accuracy                           0.45      1000
   macro avg       0.48      0.44      0.41      1000
weighted avg       0.47      0.45      0.42      1000



In [81]:
clf = LogisticRegression(C=0.1, class_weight='balanced')
clf.fit(X, y)

preds = clf.predict(X_test)

print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

           0       0.39      0.41      0.40       341
           1       0.40      0.42      0.41       319
           2       0.51      0.46      0.48       340

    accuracy                           0.43      1000
   macro avg       0.43      0.43      0.43      1000
weighted avg       0.43      0.43      0.43      1000

